In [125]:
# Import libraries 
import pandas as pd
import requests
from scipy.optimize import minimize
import pulp
import numpy as np

# Extract Data

In [126]:
# FPL API endpoint
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(url)

# Check if the response is successful
if response.status_code == 200:
    print("Successfull request")
    data = response.json()

Successfull request


In [127]:
# Create team dictionary
team_dict={}
for team in data['teams']:
    
    team_code=team['code']
    team_name=team['name']    
    
    team_dict[team_code]=team_name

# Transform Data

In [128]:
# Initialize a dataframe to store player details
df = pd.DataFrame(columns=['team','position','first_name','last_name', 'web_name','ict_index', 'points', 'price'])

In [129]:
# Create player type function
def element_type(el):
    
    if el ==1:
        return 'GKP'
    
    elif el ==2:
        return 'DEF'
    
    elif el == 3:
        
        return 'MID'
    
    elif el ==4:
        
        return 'FWD'

In [130]:
# Add each player to the dataframe
for player in players:
    
    # Get attributes
    first_name=player['first_name']
    last_name=player['second_name']
    web_name=player['web_name']
    team_code=player['team_code']
    team = team_dict[team_code]
    points=player['total_points']
    ict_index=player['ict_index']      
    cost=player['now_cost']/10
    position=element_type(player['element_type'])
    
    # Put them into new row of table
    df.loc[len(df)] = [team, position, first_name, last_name, web_name, ict_index, points, cost]

# Add in ID column
df['id'] = df.index
# Move to beginning
df = df[ ['id'] + [ col for col in df.columns if col != 'id' ] ]

In [131]:
# Create boolean indicators for each position
df['gkp']=df['position'].apply(lambda x: 1 if x=='GKP' else 0)
df['def']=df['position'].apply(lambda x: 1 if x=='DEF' else 0)
df['mid']=df['position'].apply(lambda x: 1 if x=='MID' else 0)
df['fwd']=df['position'].apply(lambda x: 1 if x=='FWD' else 0)

In [132]:
df.head()

,id,team,position,first_name,last_name,web_name,ict_index,points,price,gkp,def,mid,fwd
0,0,Arsenal,MID,Fábio,Ferreira Vieira,Fábio Vieira,0.0,0,5.4,0,0,1,0
1,1,Arsenal,FWD,Gabriel,Fernando de Jesus,G.Jesus,52.6,42,6.6,0,0,0,1
2,2,Arsenal,DEF,Gabriel,dos Santos Magalhães,Gabriel,94.0,94,6.3,0,1,0,0
3,3,Arsenal,FWD,Kai,Havertz,Havertz,142.6,96,7.8,0,0,0,1
4,4,Arsenal,GKP,Karl,Hein,Hein,0.0,0,4.0,1,0,0,0


# Write optimization problem

**Objective Function**

f(x) = sum(points)

**Constraints**

1. Max budget = 100.0
2. Exactly 2 gks, 5 defenders, 5 midfielders, 3 forwards
3. Max 3 players per team

In [133]:
# Extract player details into lists
players_name=list(df['web_name'])
points=list(df['points'])
price=list(df['price'])
positions=list(df['position'])
team=list(df['team'])
gkp=list(df['gkp'])
defender=list(df['def'])
mid=list(df['mid'])
fwd=list(df['fwd'])
num_players=len(df)

# Define fpl budget to spend
total_budget=100

In [134]:
# Initialise the optimization problem as a maximization or minimization problem
model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)

# Assign a decision variable to each player
decisions = [pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer') 
             for i in range(len(df))]

/opt/anaconda3/envs/ml_env/lib/python3.12/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [135]:
# Defining the objective function
model += sum((decisions[i]) * points[i] for i in range(num_players)), "Objective"

In [136]:
# cost contraint
model += sum((decisions[i]) * price[i] for i in range(num_players)) <= total_budget  # total cost

# 2  goalkeeper
model += sum(decisions[i] for i in range(num_players) if positions[i] == 'GKP') == 2

# 5 total defenders
model += sum(decisions[i] for i in range(num_players) if positions[i] == 'DEF') == 5

# 5 total midfielders
model += sum(decisions[i] for i in range(num_players) if positions[i] == 'MID') == 5

# 3 total forwards
model += sum(decisions[i] for i in range(num_players) if positions[i] == 'FWD') == 3

# club constraint
for team_name in df['team'].unique():
    model += sum(decisions[i] for i in range(len(df)) if team[i] == team_name) <= 3  # max 3 players per team
    
# total players constraint    
model += sum(decisions) == 15

In [137]:
# Solve the optimization problem
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/ml_env/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/wq/zrb190x53n399ty_pfx_v6300000gn/T/951ed13632b04b59b18170b10b8186d5-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/wq/zrb190x53n399ty_pfx_v6300000gn/T/951ed13632b04b59b18170b10b8186d5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 31 COLUMNS
At line 5181 RHS
At line 5208 BOUNDS
At line 5981 ENDATA
Problem MODEL has 26 rows, 772 columns and 3068 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1860 - 0.00 seconds
Cgl0003I 0 fixed, 2 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 26 rows, 681 columns (681 integer (626 of which binary)) and 2704 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial stat

1

In [138]:
# Define variables
price_sum=0
points_sum=0
goalkeepers=[]
defenders=[]
midfielders=[]
forwards=[]

# Sort players by position
for i in range(len(df)):
    
    if decisions[i].value() == 1:
        
        if positions[i] == 'GKP':
            goalkeepers.append(i)
        
        elif positions[i] == 'DEF':
            defenders.append(i)
        
        elif positions[i] == 'MID':
            midfielders.append(i)
            
        elif positions[i] == 'FWD':
            forwards.append(i)
        
        else:
            print("No position found")

# Print players
print("============ Players ============\n")
for i in goalkeepers:
        print(positions[i], players_name[i], price[i])
        price_sum+=price[i]
        points_sum+=points[i]
        
for i in defenders:
        print(positions[i], players_name[i], price[i])
        price_sum+=price[i]
        points_sum+=points[i]
        
for i in midfielders:
        print(positions[i], players_name[i], price[i])
        price_sum+=price[i]
        points_sum+=points[i]
        
for i in forwards:
        print(positions[i], players_name[i], price[i])
        price_sum+=price[i]
        points_sum+=points[i]
        
        
print (f"\nPrice sum: {round(price_sum,3)}")
print (f"Points sum: {points_sum}")

============ Players ============

GKP Pickford 5.1
GKP Sels 5.0
DEF Kerkez 5.0
DEF Muñoz 4.8
DEF Robinson 5.1
DEF Hall 5.1
DEF Aina 5.4
MID Kluivert 5.9
MID Mbeumo 7.9
MID Palmer 11.2
MID M.Salah 13.7
MID Amad 5.6
FWD Wissa 6.3
FWD Wood 7.1
FWD Cunha 6.8

Price sum: 100.0
Points sum: 1847


# Write optimization problem (with additional constraints)

**Objective Function**

f(x) = sum(points)

**Constraints**

1. Max budget = 100.0
2. Exactly 2 gks, 5 defenders, 5 midfielders, 3 forwards
3. Max 3 per team
4. Starter constraints (new)
5. Captain constraints (new)

In [139]:
# Extract player details into lists
players_name=list(df['web_name'])
points=list(df['points'])
price=list(df['price'])
positions=list(df['position'])
team=list(df['team'])
gkp=list(df['gkp'])
defender=list(df['def'])
mid=list(df['mid'])
fwd=list(df['fwd'])

num_players=len(df)
total_budget=100

In [140]:
model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)

# Starters decision var.
decisions = [pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer') 
             for i in range(len(df))]

# Captain decision var.
captain_decisions = [
        pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(len(df))]

# Bench decision var.
bench_decisions = [
        pulp.LpVariable("z{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]

In [141]:
# Objective function now includes 3 decision variables
model += sum((captain_decisions[i] + decisions[i] + bench_decisions[i]*0.3) * points[i] for i in range(num_players)), "Objective"

In [142]:
# Budget constraint
model += sum((decisions[i] + bench_decisions[i]) * price[i] for i in range(num_players)) <= total_budget  # total cost

In [143]:
# 1 starting goalkeeper
model += sum(decisions[i] for i in range(num_players) if positions[i] == 'GKP') == 1
# 2 total goalkeepers
model += sum(decisions[i] + bench_decisions[i] for i in range(num_players) if positions[i] == 'GKP') == 2

# 3-5 starting defenders
model += sum(decisions[i] for i in range(num_players) if positions[i] == 'DEF') >= 3
model += sum(decisions[i] for i in range(num_players) if positions[i] == 'DEF') <= 5
# 5 total defenders
model += sum(decisions[i] + bench_decisions[i] for i in range(num_players) if positions[i] == 'DEF') == 5

# 3-5 starting midfielders
model += sum(decisions[i] for i in range(num_players) if positions[i] == 'MID') >= 3
model += sum(decisions[i] for i in range(num_players) if positions[i] == 'MID') <= 5

# 5 total midfielders
model += sum(decisions[i] + bench_decisions[i] for i in range(num_players) if positions[i] == 'MID') == 5

# 1-3 starting attackers
model += sum(decisions[i] for i in range(num_players) if positions[i] == 'FWD') >= 1
model += sum(decisions[i] for i in range(num_players) if positions[i] == 'FWD') <= 3

# 3 total attackers
model += sum(decisions[i] + bench_decisions[i] for i in range(num_players) if positions[i] == 'FWD') == 3

In [144]:
# club constraint
for team_name in df['team'].unique():
    model += sum(decisions[i] + bench_decisions[i] for i in range(len(df)) if team[i] == team_name) <= 3  # max 3 players per team


In [145]:
# captain and bench constraint
model += sum(captain_decisions) == 1  # 1 captain
model += sum(decisions) == 11  # total players
model += sum(bench_decisions) == 4  # total players

for i in range(len(df)):  # captain must also be on team
    model += (decisions[i] - captain_decisions[i]) >= 0  # captain must also be on team
    model += (decisions[i] + bench_decisions[i]) <= 1  # subs must not be on team

In [146]:
# Solve optimizaton provlem
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/ml_env/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/wq/zrb190x53n399ty_pfx_v6300000gn/T/0c34ec96aca748a6a2f92c2a6d7d74df-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/wq/zrb190x53n399ty_pfx_v6300000gn/T/0c34ec96aca748a6a2f92c2a6d7d74df-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1584 COLUMNS
At line 19248 RHS
At line 20828 BOUNDS
At line 23145 ENDATA
Problem MODEL has 1579 rows, 2316 columns and 11420 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1834.2 - 0.00 seconds
Cgl0004I processed model has 1576 rows, 2316 columns (2316 integer (2316 of which binary)) and 10748 elements
Cutoff increment increased from 1e-05 to 0.0999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.5
Cbc0038I Pass   1: su

1

In [147]:
# Define variables
price_sum=0
points_sum=0
goalkeepers=[]
defenders=[]
midfielders=[]
forwards=[]

# Sort players
for i in range(len(df)):
    
    if decisions[i].value() == 1:
        
        if positions[i] == 'GKP':
            goalkeepers.append(i)
        
        elif positions[i] == 'DEF':
            defenders.append(i)
        
        elif positions[i] == 'MID':
            midfielders.append(i)
            
        elif positions[i] == 'FWD':
            forwards.append(i)
        
        else:
            print("No position found")

# Print players
print("============ Starters ============\n")
for i in goalkeepers:
        print(positions[i], players_name[i], price[i])
        price_sum+=price[i]
        points_sum+=points[i]
        
for i in defenders:
        print(positions[i], players_name[i], price[i])
        price_sum+=price[i]
        points_sum+=points[i]
        
for i in midfielders:
        print(positions[i], players_name[i], price[i])
        price_sum+=price[i]
        points_sum+=points[i]
        
for i in forwards:
        print(positions[i], players_name[i], price[i])
        price_sum+=price[i]
        points_sum+=points[i]


# Define variables
goalkeepers=[]
defenders=[]
midfielders=[]
forwards=[]

# Sort players
for i in range(len(df)):
    
    if bench_decisions[i].value() == 1:
        
        if positions[i] == 'GKP':
            goalkeepers.append(i)
        
        elif positions[i] == 'DEF':
            defenders.append(i)
        
        elif positions[i] == 'MID':
            midfielders.append(i)
            
        elif positions[i] == 'FWD':
            forwards.append(i)
        
        else:
            print("No position found")

# Print players
print("\n============ Bench ============\n")
for i in goalkeepers:
        print(positions[i], players_name[i], price[i])
        price_sum+=price[i]
        points_sum+=points[i]
        
for i in defenders:
        print(positions[i], players_name[i], price[i])
        price_sum+=price[i]
        points_sum+=points[i]
        
for i in midfielders:
        print(positions[i], players_name[i], price[i])
        price_sum+=price[i]
        points_sum+=points[i]
        
for i in forwards:
        print(positions[i], players_name[i], price[i])
        price_sum+=price[i]
        points_sum+=points[i]
        
for i in range(len(df)):
    
    if captain_decisions[i].value() == 1:      
         print("\nCAPTAIN:",players_name[i])
        
print (f"\nPrice sum: {price_sum}")
print (f"Points sum: {points_sum}")

============ Starters ============

GKP Pickford 5.1
DEF Hall 5.1
DEF Aina 5.4
DEF Milenković 4.8
MID Kluivert 5.9
MID Mbeumo 7.9
MID Palmer 11.2
MID M.Salah 13.7
MID Amad 5.6
FWD Isak 9.5
FWD Wood 7.1

============ Bench ============

GKP Henderson 4.5
DEF Guéhi 4.6
DEF Mykolenko 4.4
FWD Strand Larsen 5.2

CAPTAIN: M.Salah

Price sum: 99.99999999999999
Points sum: 1818


In [148]:
print(f"Price sum: {round(price_sum,3)}")
print(f"Points sum: {points_sum}")

Price sum: 100.0
Points sum: 1818
